## Import packages

In [1]:
import pandas as pd
from CFModel import CFModel
from CFModel import DeepModel

/Users/joujun/.pyenv/versions/2.7.10/envs/please_my_fridge/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Define constants


In [4]:
RATINGS_CSV_FILE = './input/rating_result.csv'
USERS_CSV_FILE = './input/user_result.csv'
RECIPE_CSV_FILE = './input/recipe_result.csv'
MODEL_WEIGHTS_FILE = 'recipe_weights.h5'
K_FACTORS = 120
TEST_USER = 1

## Load Recipe data

In [7]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep=',', encoding='utf-8', usecols=['id', 'userid', 'recipeid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max() + 1
max_recipes = ratings['recipeid'].drop_duplicates().max() + 1
print len(ratings), 'ratings loaded.'

1048575 ratings loaded.


In [9]:
users = pd.read_csv(USERS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'nickname'])
print len(users), 'descriptions of', max_userid, 'users loaded.'

75061 descriptions of 12936 users loaded.


In [11]:
recipes = pd.read_csv(RECIPE_CSV_FILE, sep=',', encoding='utf-8', usecols=['recipeid', 'title', 'categories'])
print len(recipes), 'descriptions of', max_recipes, 'recipes loaded.'

ValueError: Usecols do not match names.

## Make recommendations for a given user

In [95]:
trained_model = DeepModel(max_userid, max_movieid, K_FACTORS)

In [96]:
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [97]:
users[users['userid'] == TEST_USER]

,userid,nickname
1,1,dislab1


In [98]:
def predict_rating(userid, recipeid):
    return trained_model.rate(userid - 1, recipeid - 1)

In [99]:
user_ratings = ratings[ratings['userid'] == TEST_USER][['userid', 'recipeid', 'rating']]

In [100]:
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(recipes, 
                                                on='recipeid', 
                                                how='inner',
                                                suffixes=['_u', '_m']
                                               ).head(10)

,userid,recipeid,rating,prediction,title,categories
0,1,982,5,0.510128,a,a
1,1,335,5,0.584509,a,a
2,1,259,5,0.648876,a,a
3,1,264,5,0.579366,a,a
4,1,270,5,0.557525,a,a
5,1,279,5,0.542460,a,a
6,1,286,5,0.673987,a,a
7,1,300,5,0.575933,a,a
8,1,304,5,0.505067,a,a
9,1,306,5,0.542020,a,a


In [101]:
recommendations = ratings[ratings['recipeid'].isin(user_ratings['recipeid']) == False][['recipeid']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(recipes,
                                                 on='recipeid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,recipeid,prediction,title,categories
0,53,0.701236,a,a
1,771,0.699286,a,a
2,2082,0.699084,a,a
3,276,0.697765,a,a
4,911,0.693182,a,a
5,348,0.692550,a,a
6,1028,0.688816,a,a
7,676,0.688490,a,a
8,253,0.687564,a,a
9,900,0.686128,a,a
